In [1]:
import os
import pandas as pd


base_dir = "./pgm"
dataframes = {}

for year in range(1990, 2022):
    year_dir = os.path.join(base_dir, f"year={year}/")
    
    if os.path.exists(year_dir):
        year_dataframes = []
        
        for file in os.listdir(year_dir):
            if file.endswith(".parquet"):
                file_path = os.path.join(year_dir, file)
                
                df = pd.read_parquet(file_path)
                
                year_dataframes.append(df)
        
        if year_dataframes:
            dataframes[year] = pd.concat(year_dataframes, ignore_index=True)



In [2]:
dataframes[1990]

KeyError: 1990

In [3]:
master_df = pd.concat(dataframes.values(), ignore_index=True)

ValueError: No objects to concatenate